In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data4.csv')

In [4]:
df.head()

,Unnamed: 0,steam_appid,name,tags
0,0,730,Counter-Strike: Global Offensive,action multiplayer steamachievements fullcontr...
1,1,570,Dota 2,action multiplayer coop steamtradingcards stea...
2,2,578080,PUBG: BATTLEGROUNDS,action multiplayer pvp onlinepvp stats remotep...
3,3,1172470,Apex Legends,action multiplayer pvp onlinepvp coop onlineco...
4,4,1446780,MONSTER HUNTER RISE,action singleplayer multiplayer coop onlinecoo...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

In [6]:
ps = PorterStemmer()

In [7]:
def stem(s):
  l = []
  for i in s.split():
    l.append(ps.stem(i))
  s = " ".join(l)
  return s

In [8]:
df['tags'] = df['tags'].apply(stem)

In [9]:
tfidf = TfidfVectorizer(max_features=3000,stop_words='english')

In [10]:
vectors = tfidf.fit_transform(df['tags']).toarray()

In [11]:
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
similarity = cosine_similarity(vectors)

In [14]:
similarity[0]

array([1.        , 0.26582132, 0.14064059, ..., 0.01518613, 0.01788006,
       0.06656336])

In [15]:
k1 = sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[:6]

k2 = sorted(list(enumerate(similarity[1])),reverse=True,key=lambda x:x[1])[:6]

In [16]:
k1 = k1[1:]
k1

[(581, 0.5064236080270216),
 (2553, 0.43331354643615694),
 (2624, 0.39226658574953654),
 (124, 0.3844687373711558),
 (811, 0.3811727449027077)]

In [17]:
k2 = k2[1:]
k2

[(229, 0.3314387371338046),
 (1050, 0.30645107150084283),
 (1834, 0.2872152783602394),
 (1415, 0.27890755193302524),
 (843, 0.2751551262349293)]

In [18]:
k = k1+k2
k

[(581, 0.5064236080270216),
 (2553, 0.43331354643615694),
 (2624, 0.39226658574953654),
 (124, 0.3844687373711558),
 (811, 0.3811727449027077),
 (229, 0.3314387371338046),
 (1050, 0.30645107150084283),
 (1834, 0.2872152783602394),
 (1415, 0.27890755193302524),
 (843, 0.2751551262349293)]

In [19]:
sorted(list(k),reverse=True,key= lambda x:x[1])

[(581, 0.5064236080270216),
 (2553, 0.43331354643615694),
 (2624, 0.39226658574953654),
 (124, 0.3844687373711558),
 (811, 0.3811727449027077),
 (229, 0.3314387371338046),
 (1050, 0.30645107150084283),
 (1834, 0.2872152783602394),
 (1415, 0.27890755193302524),
 (843, 0.2751551262349293)]

In [20]:
actual_ratings_df = pd.DataFrame({
    'user_id': [0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
    'steam_appid': [292030, 271590, 424840, 50130, 7670, 1180660, 393380, 205100, 377160, 72850, 412020, 307690, 271590, 532210, 108710, 1091500, 292030, 534380, 225540, 552520, 244210, 284160, 227300, 242050, 418370, 1222140, 730, 570, 578080, 1172470, 271590, 346110, 252490, 431960, 1086940, 221100, 739630, 220240, 648800, 220240, 892970, 221380, 221380, 333420, 945360, 571740, 40970, 105600, 413150, 1557740, 266840, 297000, 4760, 393380],
    'actual rating': [9.0, 9.0, 10.0, 5.0, 2.0, 7.5, 8.0, 2.0, 5.0, 5.0, 8.0, 6.0, 6.0, 7.5, 5.0, 5.0, 5.0, 8.0, 4.0, 4.0, 7.5, 7.5, 7.5, 5.0, 2.0, 6.0, 4.0, 1.0, 2.0, 2.0, 5.0, 4.0, 5.0, 9.0, 6.0, 9.0, 5.0, 0.0, 1.5, 3.5, 7.5, 9.0, 3.0, 3.5, 5.0, 4.0, 5.0, 4.0, 2.0, 4.0, 5.0, 0.0, 3.0, 5.0]
})

data3 = pd.read_csv('data3.csv')
actual_ratings_with_names = pd.merge(actual_ratings_df, data3, on='steam_appid')

columns_to_keep = ['user_id', 'steam_appid', 'actual rating', 'name']
actual_ratings_with_names = actual_ratings_with_names[columns_to_keep]

In [21]:
desired_steam_appid = 730

index = df[df['steam_appid'] == desired_steam_appid].index[0]

game_similarity_scores = similarity[index]

sorted_similarity_indices = sorted(range(1, len(game_similarity_scores)), key=lambda i: game_similarity_scores[i], reverse=True)

top_6_similarity_scores = [game_similarity_scores[i] for i in sorted_similarity_indices[:6]]

print(top_6_similarity_scores)

[0.5064236080270216, 0.43331354643615694, 0.39226658574953654, 0.3844687373711558, 0.3811727449027077, 0.3432071800364354]


In [22]:
import numpy as np

desired_steam_appid = 730
index = df[df['steam_appid'] == desired_steam_appid].index[0]

game_similarity_scores = similarity[index]

sorted_similarity_indices = sorted(range(1, len(game_similarity_scores)), key=lambda i: game_similarity_scores[i], reverse=True)

top_6_similarity_scores = [game_similarity_scores[i] for i in sorted_similarity_indices[:6]]

average_similarity = np.mean(top_6_similarity_scores)

weighted_similarity = average_similarity * 10

print("Weighted similarity score:", weighted_similarity)

Weighted similarity score: 4.068087337538357


In [23]:
import numpy as np

steam_appids = [292030, 271590, 424840, 50130, 7670, 1180660, 393380, 205100, 377160, 72850, 412020, 307690, 271590, 532210, 108710, 1091500, 292030, 534380, 225540, 552520, 244210, 284160, 227300, 242050, 418370, 1222140, 730, 570, 578080, 1172470, 271590, 346110, 252490, 431960, 1086940, 221100, 739630, 220240, 648800, 220240, 892970, 221380, 221380, 333420, 945360, 571740, 40970, 105600, 413150, 1557740, 266840, 297000, 4760, 393380]
predicted_rating = []

for appid in steam_appids:
    index = df[df['steam_appid'] == appid].index[0]
    
    game_similarity_scores = similarity[index]
    
    sorted_similarity_indices = sorted(range(1, len(game_similarity_scores)), key=lambda i: game_similarity_scores[i], reverse=True)
    
    top_6_similarity_scores = [game_similarity_scores[i] for i in sorted_similarity_indices[:6]]
    
    average_similarity = np.mean(top_6_similarity_scores)
    
    weighted_similarity = average_similarity * 10
    
    predicted_rating.append(weighted_similarity)

In [24]:
predicted_ratings_df = pd.DataFrame({
    'user_id': [0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
    'steam_appid': [292030, 271590, 424840, 50130, 7670, 1180660, 393380, 205100, 377160, 72850, 412020, 307690, 271590, 532210, 108710, 1091500, 292030, 534380, 225540, 552520, 244210, 284160, 227300, 242050, 418370, 1222140, 730, 570, 578080, 1172470, 271590, 346110, 252490, 431960, 1086940, 221100, 739630, 220240, 648800, 220240, 892970, 221380, 221380, 333420, 945360, 571740, 40970, 105600, 413150, 1557740, 266840, 297000, 4760, 393380],
    'predicted rating': predicted_rating
})

data3 = pd.read_csv('data3.csv')
predicted_ratings_with_names = pd.merge(predicted_ratings_df, data3, on='steam_appid')

columns_to_keep = ['user_id', 'steam_appid', 'predicted rating', 'name']
predicted_ratings_with_names = predicted_ratings_with_names[columns_to_keep]

In [26]:
final_df = pd.merge(actual_ratings_with_names, predicted_ratings_with_names, on=['user_id', 'steam_appid'])

columns_to_keep = ['user_id', 'steam_appid', 'actual rating', 'predicted rating', 'name_x']
final_df = final_df[columns_to_keep]

final_df = final_df.rename(columns={
    'name_x': 'name'
})

final_df.head(7)

,user_id,steam_appid,actual rating,predicted rating,name
0,0,292030,9.0,4.640919,The Witcher 3: Wild Hunt
1,1,271590,9.0,3.980669,Grand Theft Auto V
2,2,424840,10.0,4.163052,Little Nightmares
3,2,50130,5.0,4.235017,Mafia II (Classic)
4,2,7670,2.0,4.454317,BioShock
5,2,1180660,7.5,3.998131,Tell Me Why
6,2,393380,8.0,4.741070,Squad


In [31]:
from sklearn.metrics import mean_absolute_error

actual_ratings = final_df['actual rating'].tolist()
predicted_ratings = final_df['predicted rating'].tolist()

mae = mean_absolute_error(actual_ratings, predicted_ratings)

print(f'MAE: {mae}')

MAE: 2.069273763487834


In [30]:
from sklearn.metrics import root_mean_squared_error

actual_ratings = final_df['actual rating'].tolist()
predicted_ratings = final_df['predicted rating'].tolist()

rmse = root_mean_squared_error(actual_ratings, predicted_ratings)

print(f"RMSE: {rmse}")

RMSE: 2.586700024179224


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity